In [1]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

In [2]:
#Switch function
def switch(x):
    return {
        'Private gathering restrictions': 'restrictions to 10 indoors, 25 outdoors',
        'Acitivies closures':'Closure of all restaurants, bars, gyms, movie theatres and casinos',
        'Control': 'Stringent measures: Broader-scale actions and restrictions across multiple sectors to control the spread. Restrictions are the most severe available before widescale business or organizational closings.',
        'Prevent': 'Standard measures: Capacity limits in various public settings where people gather. Highest-risk settings stay closed.',
        'Protect': 'Strengthened measures: Stronger targeted enforcement, fines and more education to limit the spread. Public health measures for high-risk settings.',
        'Restrict': 'Intermediate measures: Stronger actions, restrictions and enforcement while avoiding closures.',
        'Lockdown': 'Maximum measures: Widescale actions and restrictions, including closures, to stop or slow the spread.?We will consider declaring a state of emergency.',
        'Stay-at-home': 'Until your region returns to the colour-coded framework, the shutdown and stay-at-home orders still apply in your area.'
    }.get(x, 'No description available')



In [3]:

ontario_measures = pd.read_csv("response_framework_raw.csv", sep=',')

ontario_measures.head()

,Reporting_PHU,Reporting_PHU_id,Status_PHU,start_date,end_date,PHU_url
0,Algoma Public Health Unit,2226,Prevent,2020-11-07T00:00:00,2020-12-25T00:00:00,http://www.algomapublichealth.com/
1,Brant County Health Unit,2227,Protect,2020-11-07T00:00:00,2020-11-15T00:00:00,http://www.bchu.org/
2,Chatham-Kent Health Unit,2240,Prevent,2020-11-07T00:00:00,2020-11-22T00:00:00,http://ckphu.com/
3,Durham Region Health Department,2230,Protect,2020-11-07T00:00:00,2020-11-15T00:00:00,https://www.durham.ca/en/health-and-wellness/h...
4,Eastern Ontario Health Unit,2258,Protect,2020-11-07T00:00:00,2020-11-15T00:00:00,http://www.eohu.ca/


In [4]:
special_measures_dimension = pd.DataFrame(columns=['speacial_measures_surrogate_key','status','region', 'description','isProvincial', 'start_date','end_date'])


In [5]:
start_date = datetime(2020, 9, 1).date()
end_date = datetime(2020, 12, 31).date()
isProvincial = False
isDone = False
for skey, row in ontario_measures.iterrows():
    matches = ["durham", "halton", "peel", "york", "toronto", "ottawa"]#list of regions
    specialDates = ["2020-09-01","2020-10-02"]


    if(not isDone):
        for curr_date in specialDates:
            if(curr_date == '2020-09-01'):
                curr_end_date = '2020-10-01'
                status = 'Private gathering restrictions'

            elif(curr_date == '2020-10-02'):
                curr_end_date = '2020-11-06'
                status = 'Acitivies closures'

            description = switch(status)#find the description for the current status
            for x in matches:
                ontario_measures_row = [skey + 100]#surrogate key
                ontario_measures_row.append(status)#title
                ontario_measures_row.append(x)#put only the region
                ontario_measures_row.append(description)
                ontario_measures_row.append(True) 
                ontario_measures_row.append(curr_date)
                ontario_measures_row.append(curr_end_date)       
                special_measures_dimension.loc[len(special_measures_dimension)] = ontario_measures_row
                skey += 1
        isDone = True

    curr_date =pd.to_datetime(row['start_date']).date() 

    phu = row['Reporting_PHU']
    matches = ["durham", "halton", "peel", "york", "toronto", "ottawa"]#list of regions

    #if(any(x in phu.lower() for x in matches) ):
    for x in matches:
        if(x in phu.lower()):

            if(curr_date >= start_date and curr_date <= end_date):

                description = switch(row['Status_PHU'])#find the description for the current status
    
                ontario_measures_row = [skey + 111]#surrogate key
                ontario_measures_row.append(row['Status_PHU'])#title
                ontario_measures_row.append(x)#put only the region
                ontario_measures_row.append(description)
                ontario_measures_row.append(isProvincial) 
                ontario_measures_row.append(curr_date)
                ontario_measures_row.append(pd.to_datetime(row['end_date']).date() )
                special_measures_dimension.loc[len(special_measures_dimension)] = ontario_measures_row


In [6]:
special_measures_dimension.head()


,speacial_measures_surrogate_key,status,region,description,isFederal,start_date,end_date
0,100,Private gathering restrictions,durham,"restrictions to 10 indoors, 25 outdoors",True,2020-09-01,2020-10-01
1,101,Private gathering restrictions,halton,"restrictions to 10 indoors, 25 outdoors",True,2020-09-01,2020-10-01
2,102,Private gathering restrictions,peel,"restrictions to 10 indoors, 25 outdoors",True,2020-09-01,2020-10-01
3,103,Private gathering restrictions,york,"restrictions to 10 indoors, 25 outdoors",True,2020-09-01,2020-10-01
4,104,Private gathering restrictions,toronto,"restrictions to 10 indoors, 25 outdoors",True,2020-09-01,2020-10-01


In [7]:
special_measures_dimension.fillna(0, inplace = True)

special_measures_dimension.to_csv("special_measures_dimension.csv",index=False)